i Q1. You are working on a machine learning project where you have a dataset containing numerical and
categorical features. You have identified that some of the features are highly correlated and there are
missing values in some of the columns. You want to build a pipeline that automates the feature
engineering process and handles the missing valuesD

i Design a pipeline that includes the following steps"
Use an automated feature selection method to identify the important features in the datasetC
Create a numerical pipeline that includes the following steps"
Impute the missing values in the numerical columns using the mean of the column valuesC
Scale the numerical columns using standardisationC
Create a categorical pipeline that includes the following steps"
Impute the missing values in the categorical columns using the most frequent value of the columnC
One-hot encode the categorical columnsC
Combine the numerical and categorical pipelines using a ColumnTransformerC
Use a Random Forest Classifier to build the final modelC
Evaluate the accuracy of the model on the test datasetD
Note! Your solution should include code snippets for each step of the pipeline, and a brief explanation of
each step. You should also provide an interpretation of the results and suggest possible improvements for
the pipelineD

In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=sns.load_dataset('tips') i load dataset

df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.shape

(244, 7)

In [5]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [6]:
df.shape

(244, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer

In [9]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [10]:

x=df.drop('time',axis=1)
y=df['time']

In [11]:
np.random.seed(42)  i For reproducibility
nan_fraction = 0.1  i Fraction of data to be replaced with NaN

i Randomly choose elements to set to NaN
for col in x.columns:
    x.loc[x.sample(frac=nan_fraction).index, col] = np.nan

In [12]:
num_cols=x.select_dtypes(exclude='category').columns
num_cols

Index(['total_bill', 'tip', 'size'], dtype='object')

In [13]:
cate_cols=x.select_dtypes(include='category').columns
cate_cols

Index(['sex', 'smoker', 'day'], dtype='object')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [15]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [16]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [18]:
num_pipeline = Pipeline(
    steps=[
        ('impute', SimpleImputer(strategy='median')),  i Handle missing values with the median of the column
        ('scale', StandardScaler())  i Apply Z-score normalization
    ]
)

# Define the categorical pipeline
cate_pipeline = Pipeline(
    steps=[
        ('impute', SimpleImputer(strategy='most_frequent')),  i Handle missing values with the mode of the column
        ('encode', OneHotEncoder())  i Convert categorical data to one-hot encoded data
    ]
)

# Combine the numerical and categorical pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),
        ('cat', cate_pipeline, cate_cols)
    ]
)

In [19]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='median')),
                                                 ('scale', StandardScaler())]),
                                 Index(['total_bill', 'tip', 'size'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encode', OneHotEncoder())]),
                                 Index(['sex', 'smoker', 'day'], dtype='object'))])

In [20]:
x_train=preprocessor.fit_transform(x_train)

In [21]:
x_test=preprocessor.transform(x_test)

In [22]:
np.array(x_train)

array([[-0.70644139, -0.73843042, -0.59350491, ...,  1.        ,
         0.        ,  0.        ],
       [-0.23033592, -1.2167944 ,  0.51524052, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.69121206,  1.65338946,  1.62398596, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.23033592, -0.11805214, -0.59350491, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.41605279,  2.25134444, -0.59350491, ...,  1.        ,
         0.        ,  0.        ],
       [-1.45542266,  1.61601728, -0.59350491, ...,  1.        ,
         0.        ,  0.        ]])

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [24]:
model=RandomForestClassifier(n_estimators=100,max_depth=10)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
    
acc=accuracy_score(y_test,y_pred)
print(acc)
print(classification_report(y_test,y_pred))
confusion_matrix(y_test,y_pred)

0.8918918918918919
              precision    recall  f1-score   support

      Dinner       0.90      0.96      0.93        56
       Lunch       0.86      0.67      0.75        18

    accuracy                           0.89        74
   macro avg       0.88      0.82      0.84        74
weighted avg       0.89      0.89      0.89        74



array([[54,  2],
       [ 6, 12]], dtype=int64)

# Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then  use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [51]:
df=sns.load_dataset('iris')

In [53]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [55]:
x=df.drop('species',axis=1)
y=df['species']

In [91]:
num_cols=x.columns

In [93]:
num_pipeline = Pipeline(
    steps=[
        ('impute', SimpleImputer(strategy='median')),  # Handle missing values with the median of the column
        ('scale', StandardScaler())  # Apply Z-score normalization
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols)
    ]
)


In [95]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

In [99]:
rm_clf=RandomForestClassifier()
log_reg=LogisticRegression()

In [103]:
clf = VotingClassifier(
    estimators=[
        ('rf', rm_clf),
        ('lr', log_reg)
    ],
    voting='hard'
)

# Create the final pipeline
model_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('voting_clf', clf)
    ]
)

In [113]:
clf.score

<bound method ClassifierMixin.score of VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('lr', LogisticRegression())])>

In [105]:
model_pipeline.fit(x_train, y_train)
y_pred = model_pipeline.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the VotingClassifier: {accuracy:.2f}')

Accuracy of the VotingClassifier: 1.00


In [107]:
print(classification_report(y_test,y_pred))
confusion_matrix(y_test,y_pred)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



array([[10,  0,  0],
       [ 0,  9,  0],
       [ 0,  0, 11]], dtype=int64)